In [21]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tensorflow as tf
# from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers


In [22]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Define parameters
IMG_SIZE = (180,180)
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5
_input = Input((180,180,3))

In [23]:
# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)


Found 4100 images belonging to 5 classes.
Found 1022 images belonging to 5 classes.


In [24]:
# Test data generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'Testing_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)


Found 1275 images belonging to 5 classes.


In [25]:
from tensorflow.keras.models import Sequential

In [8]:
# # from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

# vgg_model = Sequential([
#     layers.Rescaling(1./255, input_shape=(180, 180, 3)),
#     layers.Conv2D(32, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(32, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
#     layers.Conv2D(64, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(64, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
#     layers.Conv2D(128, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(128, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
#     layers.Conv2D(512, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(512, 3, padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
#     layers.Flatten(),
#     layers.Dense(units=1024, activation='relu'),
#     layers.Dense(units=5, activation='sigmoid')
# ])

In [26]:

model = tf.keras.models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(rate=0.15),  # Adding dropout regularization to deal with overfitting
    # The second convolution
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(rate=0.1),
    # The third convolution
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(rate=0.10),
    # Flatten the results to feed into a DNN
    layers.Flatten(),
    # 512 neuron hidden layer
    layers.Dense(128, activation='relu'),
    # 3 output neurons for the 3 classes of Animal Images
    layers.Dense(5, activation='softmax')
])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# # Print the model summary
# model.summary()


In [27]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [28]:
# epochs=25
history =model.fit(train_generator,validation_data=validation_generator,epochs=30,batch_size=32)

Epoch 1/30
129/129 [==============================] - 36s 277ms/step - loss: 1.4806 - accuracy: 0.5037 - val_loss: 1.3681 - val_accuracy: 0.5254
Epoch 2/30
129/129 [==============================] - 36s 280ms/step - loss: 1.2256 - accuracy: 0.5220 - val_loss: 1.4284 - val_accuracy: 0.5254
Epoch 3/30
129/129 [==============================] - 36s 279ms/step - loss: 1.1991 - accuracy: 0.5285 - val_loss: 1.3449 - val_accuracy: 0.5333
Epoch 4/30
129/129 [==============================] - 38s 290ms/step - loss: 1.1709 - accuracy: 0.5351 - val_loss: 1.3461 - val_accuracy: 0.5294
Epoch 5/30
129/129 [==============================] - 36s 276ms/step - loss: 1.1560 - accuracy: 0.5380 - val_loss: 1.1765 - val_accuracy: 0.5450
Epoch 6/30
129/129 [==============================] - 36s 275ms/step - loss: 1.1285 - accuracy: 0.5512 - val_loss: 1.2275 - val_accuracy: 0.5352
Epoch 7/30
129/129 [==============================] - 36s 277ms/step - loss: 1.1123 - accuracy: 0.5590 - val_loss: 1.3140 - val_ac

In [29]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f"Test accuracy: {test_accuracy:.4f}")

39/39 [==============================] - 15s 381ms/step - loss: 0.8678 - accuracy: 0.6546
Test accuracy: 0.6546


In [30]:
model.save('cnn1_tf.h5')

In [9]:
from tensorflow.keras.optimizers import Adam


vgg_model.compile(optimizer=Adam(), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [11]:
# epochs=25
history =vgg_model.fit(train_generator,validation_data=validation_generator,epochs=30,batch_size=32)

Epoch 1/30


C:\Users\kk993\anaconda3\envs\my_env\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/dense/MatMul/MatMul_1' defined at (most recent call last):
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\kk993\anaconda3\envs\my_env\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Us

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f"Test accuracy: {test_accuracy:.4f}")